In [1]:
import pandas as pd

# file paths
input_file = r"C:\Users\razhd\Downloads\Mike Pence - to clean.xlsx"
output_file = r"C:\Users\razhd\Downloads\Mike Pence - columns cleaned.xlsx"

# sheets to process
sheets_to_process = ["before visit", "visit day", "after visit"]

# columns and their new names
desired_columns = [
    "url",
    "published",
    "title",
    "content",
    "source_type",
    "post_type",
    "engagement",
    "matched_profile",
    "extra_author_attributes.name",
    "source_extended_attributes.twitter_followers",
    "source_extended_attributes.instagram_followers",
    "לינק",
    "סנטימנט",
    "התייחסות לביקור",
    "הערות (האם חריג? האם יש תמונה/סרטון מהסיור?)"
]

new_column_names = {
    "url": "URL",
    "published": "Published Date",
    "title": "Title",
    "content": "Content",
    "source_type": "Source_Type",
    "post_type": "Content_Type",
    "engagement": "Engagement",
    "matched_profile": "Keyword in Post",
    "extra_author_attributes.name": "Author Name",
    "source_extended_attributes.twitter_followers": "Twitter Followers",
    "source_extended_attributes.instagram_followers": "Instagram Followers",
    "לינק": "לינק",
    "סנטימנט": "סנטימנט",
    "התייחסות לביקור": "התייחסות לביקור",
    "הערות (האם חריג? האם יש תמונה/סרטון מהסיור?)":"הערות (האם חריג? האם יש תמונה/סרטון מהסיור?)"
}

# dictionary to store processed sheets
processed_sheets = {}

# Loop through each sheet
for sheet in sheets_to_process:
    df = pd.read_excel(input_file, sheet_name=sheet)
    df = df[desired_columns]
    df = df.rename(columns=new_column_names)
    processed_sheets[sheet] = df

# save processed sheets
with pd.ExcelWriter(output_file) as writer:
    for sheet_name, df in processed_sheets.items():
        df.to_excel(writer, sheet_name=sheet_name, index=False)

print(f"Step 1: File saved successfully to {output_file}")


Step 1: File saved successfully to C:\Users\razhd\Downloads\Mike Pence - columns cleaned.xlsx


In [2]:
import pandas as pd

# file paths
input_file = r"C:\Users\razhd\Downloads\Mike Pence - columns cleaned.xlsx"
output_file = r"C:\Users\razhd\Downloads\Mike Pence - Cleaned Source Type.xlsx"

# load sheets
sheets = pd.read_excel(input_file, sheet_name=None)  # Load all sheets as a dictionary

# dictionary to store processed sheets
processed_sheets = {}

# loop through each sheet
for sheet_name, df in sheets.items():
    # Clean the Source Type column if it exists
    if "Source_Type" in df.columns:
        df["Source_Type"] = df["Source_Type"].replace({
            "SOCIALMEDIA,SOCIALMEDIA_INSTAGRAM": "Instagram",
            "SOCIALMEDIA,SOCIALMEDIA_TWITTER": "Twitter"
        })
    processed_sheets[sheet_name] = df

# Save the processed sheets to a new Excel file
with pd.ExcelWriter(output_file) as writer:
    for sheet_name, df in processed_sheets.items():
        df.to_excel(writer, sheet_name=sheet_name, index=False)

print(f"File saved successfully to {output_file}")


File saved successfully to C:\Users\razhd\Downloads\Mike Pence - Cleaned Source Type.xlsx


In [3]:
import pandas as pd
import re

# File paths
input_file = r"C:\Users\razhd\Downloads\Mike Pence - Cleaned Source Type.xlsx"
output_file = r"C:\Users\razhd\Downloads\Mike Pence - Keywords Added.xlsx"

# Specify the sheets to process
sheets_to_process = ["before visit","visit day", "after visit"]

# Function to extract keywords from the "matched_profile" column
def extract_keywords(text):
    if pd.isna(text):
        return ""
    
    keywords = []
    # Split by comma to get individual parts
    parts = str(text).split(',')
    
    # Look for the pattern "KPMG / keyword"
    for part in parts:
        if 'מילות מפתח - סיורים - KPMG /' in part:
            # Extract the keyword after the last forward slash
            keyword = part.split('/')[-1].strip()
            if keyword:
                keywords.append(keyword)
    
    # Remove duplicates while preserving order
    seen = set()
    unique_keywords = []
    for keyword in keywords:
        if keyword not in seen:
            seen.add(keyword)
            unique_keywords.append(keyword)
    
    return ", ".join(unique_keywords)

# Dictionary to store processed sheets
processed_sheets = {}

# Loop through each sheet
for sheet in sheets_to_process:
    # Read the sheet
    df = pd.read_excel(input_file, sheet_name=sheet)
    
    # Clean and extract keywords from the matched_profile column
    if "matched_profile" in df.columns:
        df["Keyword in Post"] = df["matched_profile"].apply(extract_keywords)
    elif "Keyword in Post" in df.columns:
        df["Keyword in Post"] = df["Keyword in Post"].apply(extract_keywords)
    
    processed_sheets[sheet] = df

# Save the processed sheets
with pd.ExcelWriter(output_file) as writer:
    for sheet_name, df in processed_sheets.items():
        df.to_excel(writer, sheet_name=sheet_name, index=False)

print(f"Step 3: File saved successfully to {output_file}")

Step 3: File saved successfully to C:\Users\razhd\Downloads\Mike Pence - Keywords Added.xlsx


In [4]:
import pandas as pd
import re

# File paths
input_file = r"C:\Users\razhd\Downloads\Mike Pence - Keywords Added.xlsx"
output_file = r"C:\Users\razhd\Downloads\Mike Pence - For Tamar.xlsx"

# Load all sheets into a dictionary
sheets = pd.read_excel(input_file, sheet_name=None)

# Dictionary to store processed sheets
processed_sheets = {}

# Loop through each sheet
for sheet_name, df in sheets.items():
    # Check if "Content" and "Source_Type" columns exist
    if "Content" in df.columns and "Source_Type" in df.columns:
        # Calculate index to insert the new column after "Content"
        content_index = df.columns.get_loc("Content") + 1

        # Define logic for "Post Type" based on the "Content" and "Source_Type" columns
        def determine_post_type(row):
            if row["Source_Type"].strip().lower() == "twitter":
                content = str(row["Content"])
                # Match 'RT' or 'QT' as separate words (case-sensitive)
                if re.search(r'\bRT\b', content):
                    return "Retweet"
                elif re.search(r'\bQT\b', content):
                    return "Quote Tweet"
                else:
                    return "Post"
            return "Post"  # Default for non-Twitter rows

        # Apply the logic and insert the "Post Type" column
        df.insert(content_index, "Post Type", df.apply(determine_post_type, axis=1))
    else:
        print(f"Warning: 'Content' or 'Source_Type' column not found in sheet '{sheet_name}'")

    # Add the processed sheet back to the dictionary
    processed_sheets[sheet_name] = df

# Save the processed sheets back to a new Excel file
with pd.ExcelWriter(output_file) as writer:
    for sheet_name, df in processed_sheets.items():
        df.to_excel(writer, sheet_name=sheet_name, index=False)

print(f"File saved successfully to {output_file}")


File saved successfully to C:\Users\razhd\Downloads\Mike Pence - For Tamar.xlsx


In [5]:
print(df["Content"].head(50))  # Print first 10 rows of Content

0     Putin is a war criminal and only understands s...
1     Traveled to Israel’s Northern Command today to...
2     Back home from Israel after four days I will n...
3     Back home from Israel after four days I will n...
4     Honored to sit down with my good friend Prime ...
5     Traveled to Israel’s Northern Command today to...
6     This is a moment where America needs to speak ...
7     Honored to sit down with my good friend Prime ...
8     It’s About Time!🇺🇸 US, UK launch retaliatory s...
9     Honored to attend a Havdalah Ceremony marking ...
10    Pence In Israel: October 7 Was ‘Not Simply A T...
11    Privileged to meet with President Isaac Herzog...
12    In the face of these widening dangers, I urge ...
13    Former US VP Mike Pence to Israel: 'The Americ...
14    Honored to attend a Havdalah Ceremony marking ...
15    Pence in Israel: Jewish State 'Our Most Cheris...
16    Israel is waging a war for its very survival a...
17    Privileged to meet with President @Isaac_H

In [6]:
print(df["Source_Type"].unique())

['Instagram' 'Twitter']
